In [1]:
from datasets import load_dataset

ds = load_dataset("Jinyan1/COLING_2025_MGT_multingual")

Generating train split:   0%|          | 0/674083 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/288894 [00:00<?, ? examples/s]

In [2]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'sub_source', 'lang', 'model', 'label', 'text'],
        num_rows: 674083
    })
    dev: Dataset({
        features: ['id', 'source', 'sub_source', 'lang', 'model', 'label', 'text'],
        num_rows: 288894
    })
})


In [3]:
import pandas

train_df = ds['train'].to_pandas()
dev_df = ds['dev'].to_pandas()

# Verify the conversion
print(train_df.head())
print(dev_df.head())

                                     id source   sub_source lang        model  \
0  808a846d-41c1-46ff-80ab-892b01bec5f3   m4gt        arxiv   en  gemma-7b-it   
1  673ec6fc-624a-4ad5-8d5f-a5950b0e5432   mage           wp   en        human   
2  3059e6e4-1724-4745-a833-cbb6bfb4acd8   mage         xsum   en   flan_t5_xl   
3  4ca1cde7-6dbd-4519-b60c-7f8c8fcd0d36    hc3      open_qa   zh       gpt-35   
4  f9cca231-ed31-45a9-a448-99245f72e32c    hc3  reddit_eli5   en       gpt-35   

   label                                               text  
0      1  This report summarizes the findings of the US ...  
1      0  I've been standing here for days now. Watching...  
2      1  Towell, 25, was knocked down twice during the ...  
3      1  我不确定你想要问的是什么，但是阿玛尼是一个奢侈品牌，它的产品包括化妆品、香水和时装。黑手党是...  
4      1  Sometimes when we eat certain types of food, o...  
                                     id source        sub_source lang   model  \
0  8a6ce8d6-a07c-4a2b-831e-db606a7d46ad   mage              

In [4]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
import numpy as np

# Ensure TensorFlow uses the GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

# Assuming you already have 'train_df' and 'dev_df' converted from DatasetDict
# Extract text, lang, and label columns from the DataFrame
train_texts = train_df['text'].tolist()
train_langs = train_df['lang'].tolist()
train_labels = train_df['label'].values

dev_texts = dev_df['text'].tolist()
dev_langs = dev_df['lang'].tolist()
dev_labels = dev_df['label'].values

# Tokenize the text input with tqdm progress tracking
max_length = 128  # Maximum length for the tokenized sequences

Num GPUs Available:  1


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
# Tokenizing train data with progress bar
train_encodings = {
    'input_ids': [],
    'attention_mask': []
}

for text in tqdm(train_texts, desc="Tokenizing train data"):
    enc = tokenizer(text, truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
    train_encodings['input_ids'].append(enc['input_ids'])
    train_encodings['attention_mask'].append(enc['attention_mask'])

# Convert lists to tensors
train_encodings['input_ids'] = tf.concat(train_encodings['input_ids'], axis=0)
train_encodings['attention_mask'] = tf.concat(train_encodings['attention_mask'], axis=0)

In [15]:
print(len(train_encodings['attention_mask']))

674083


In [8]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
import numpy as np

max_length = 128  # Maximum length for the tokenized sequences


# Tokenizing dev data with progress bar
dev_encodings = {
    'input_ids': [],
    'attention_mask': []
}

for text in tqdm(dev_texts, desc="Tokenizing dev data"):
    enc = tokenizer(text, truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
    dev_encodings['input_ids'].append(enc['input_ids'])
    dev_encodings['attention_mask'].append(enc['attention_mask'])

# Convert lists to tensors
dev_encodings['input_ids'] = tf.concat(dev_encodings['input_ids'], axis=0)
dev_encodings['attention_mask'] = tf.concat(dev_encodings['attention_mask'], axis=0)



Tokenizing dev data: 100%|██████████| 288894/288894 [46:33<00:00, 103.43it/s]


In [5]:
import pickle

with open('/kaggle/input/train-ids/input_ids.pkl', 'rb') as file:
    train_input_ids = pickle.load(file)
with open('/kaggle/input/train-attention/attention_masks.pkl', 'rb') as file:
    train_encodings_attention = pickle.load(file)
with open('/kaggle/input/dev-attention/dev_attention.pkl', 'rb') as file:
    dev_encodings_attention = pickle.load(file)
with open('/kaggle/input/dev-input-ids/dev_input_ids.pkl', 'rb') as file:
    dev_input_ids = pickle.load(file)

In [6]:
from sklearn.preprocessing import OneHotEncoder

# One-hot encode the language input (using sklearn OneHotEncoder)
encoder = OneHotEncoder(sparse = False)
train_langs_encoded = encoder.fit_transform(np.array(train_langs).reshape(-1, 1))
dev_langs_encoded = encoder.transform(np.array(dev_langs).reshape(-1, 1))

num_languages = train_langs_encoded.shape[1]

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [7]:
import tensorflow as tf
from transformers import TFBertModel

# Define BERT model with attention masks
bert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

# Input layers
input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

# Get BERT embeddings using input_ids and attention_mask
bert_outputs = bert_model(input_ids, attention_mask=attention_mask)
text_embeddings = tf.keras.layers.GlobalAveragePooling1D()(bert_outputs.last_hidden_state)

# Language input (one-hot encoded)
lang_input = tf.keras.layers.Input(shape=(num_languages,), name='language')

# Concatenate text embeddings with language input
concat = tf.keras.layers.Concatenate()([text_embeddings, lang_input])

# Classification head
dense = tf.keras.layers.Dense(128, activation='relu')(concat)
output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)  # Binary classification

# Define the model
model = tf.keras.Model(inputs=[input_ids, attention_mask, lang_input], outputs=output)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [8]:
history = model.fit(
    [train_input_ids,train_encodings_attention, train_langs_encoded],  # Inputs: text + language
    train_labels,  # Labels
    validation_data=([dev_input_ids, dev_encodings_attention, dev_langs_encoded], dev_labels),  # Validation
    epochs=1,
    batch_size=32
)

I0000 00:00:1727127081.878221     127 service.cc:145] XLA service 0x7cf251aa6340 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727127081.878270     127 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1727127082.046133     127 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


21066/21066 [==============================] - 12572s 593ms/step - loss: 0.1937 - accuracy: 0.9175 - val_loss: 0.1912 - val_accuracy: 0.9228


In [16]:
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer
from sklearn.preprocessing import OneHotEncoder

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# External test data: manually written sentences and their corresponding language codes
test_texts = ["by the way i feel i am not well", "OpenCellular (OC) ist eine Open-Source-Zugangsplattform mit Schwerpunkt auf ländlicher Konnektivität"]  # Example test sentences (English and German)
test_langs = ["en", "de"]  # Corresponding language codes

# Step 1: Tokenize the text input (manually written sentences) for BERT
max_length = 128  # Ensure this matches the max_length used during training
test_encodings = tokenizer(
    test_texts,
    max_length=max_length,
    padding='max_length',  # This ensures all sequences are padded to max_length
    truncation=True,  # Truncates sentences longer than max_length
    return_tensors="tf"  # Returns the tensors needed for the model
)

# Extract tokenized input IDs and attention masks
test_input_ids = test_encodings['input_ids']  # Shape: (batch_size, max_length)
test_attention_mask = test_encodings['attention_mask']  # Shape: (batch_size, max_length)

# Step 2: One-hot encode the language input
encoder = OneHotEncoder(sparse=False)  # If not already fitted, load from training step
# Fit encoder if it wasn't fitted earlier
encoder.fit(np.array(train_langs).reshape(-1, 1))  # Fitting it on the training languages, if needed

# One-hot encode the manually written language inputs
test_langs_encoded = encoder.transform(np.array(test_langs).reshape(-1, 1))  # Shape: (batch_size, num_languages)

# Print shapes for debugging
print(f"test_input_ids shape: {test_input_ids.shape}")  # Should be (batch_size, max_length)
print(f"test_attention_mask shape: {test_attention_mask.shape}")  # Should be (batch_size, max_length)
print(f"test_langs_encoded shape: {test_langs_encoded.shape}")  # Should be (batch_size, num_languages)

# Step 3: Load the trained model

# Step 4: Ensure all inputs are compatible for prediction
# The model expects [input_ids, attention_mask, one-hot encoded language input]

# Check if the batch size is correct
if test_input_ids.shape[0] == test_attention_mask.shape[0] == test_langs_encoded.shape[0]:
    # Step 5: Make predictions
    test_predictions = model.predict([test_input_ids, test_attention_mask, test_langs_encoded])
    
    # Step 6: Interpret predictions
    # Assuming binary classification (0 or 1)
    predicted_labels = (test_predictions > 0.5).astype(int)

    # Print predictions for each test sentence
    for i, (text, lang, prediction) in enumerate(zip(test_texts, test_langs, predicted_labels)):
        label = 'Machine-Generated' if prediction == 1 else 'Human-Generated'
        print(f"Text: {text}")
        print(f"Language: {lang}")
        print(f"Predicted Label: {label}\n")
else:
    print("Input shapes are incompatible for prediction. Please check the input data.")


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


test_input_ids shape: (2, 128)
test_attention_mask shape: (2, 128)
test_langs_encoded shape: (2, 9)
1/1 [==============================] - 0s 86ms/step
Text: by the way i feel i am not well
Language: en
Predicted Label: Human-Generated

Text: OpenCellular (OC) ist eine Open-Source-Zugangsplattform mit Schwerpunkt auf ländlicher Konnektivität
Language: de
Predicted Label: Machine-Generated

